<a href="https://colab.research.google.com/github/cahya-wirawan/luganda-asr/blob/main/Luganda_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!free -h

Wed Jan 19 05:08:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Preparation of the Luganda ASR  source code 

Clone the source code from https://github.com/cahya-wirawan/luganda-asr.git

In [2]:
!git clone https://github.com/cahya-wirawan/luganda-asr.git

Cloning into 'luganda-asr'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 61 (delta 24), reused 39 (delta 10), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [3]:
%cd /content/luganda-asr
!git pull

/content/luganda-asr
Already up to date.


Install the requirements

In [4]:
%%capture

!pip install -r requirements.txt

## Data Preparation

We prepare the training and test data from Google Drive directory with following file structure:

/content/drive/MyDrive/Luganda
- cv-corpus-7.0-2021-07-21-lg.tar.gz
- SampleSubmission.csv
- test_audio.zip
- Test.csv


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
luganda_data_dir = "/content/drive/MyDrive/Luganda"
luganda_cv_corpus = f"{luganda_data_dir}/cv-corpus-7.0-2021-07-21-lg.tar.gz"
luganda_test_audio_file = f"{luganda_data_dir}/test_audio.zip"
luganda_test_file = f"{luganda_data_dir}/Test.csv"

!ls -l $luganda_data_dir

total 1914072
-rw------- 1 root root 1722860000 Jan 18 18:05 cv-corpus-7.0-2021-07-21-lg.tar.gz
-rw------- 1 root root      91886 Jan 18 16:41 SampleSubmission.csv
-rw------- 1 root root  236764554 Jan 18 16:39 test_audio.zip
-rw------- 1 root root     292195 Jan 18 16:29 Test.csv


In [7]:

%%capture

# Uncompressing the test_audio.zip and cv-corpus-7.0-2021-07-21-lg.tar.gz to the 
# directory /content/data.
# It takes around 40s

%cd /content/

## clean up the directory /content/data/
!rm -rf /content/data

!mkdir -p /content/data/zindi
%cd /content/data/zindi
!unzip -o $luganda_test_audio_file && cp $luganda_test_file .
%cd /content/data
!tar xvzf $luganda_cv_corpus 

In [8]:
!echo "Content of /content/data:" && ls -l /content/data
!echo
!echo "Content of /content/data/zindi:" && ls -l /content/data/zindi

Content of /content/data:
total 8
drwxr-xr-x 3 root root 4096 Jan 19 05:12 cv-corpus-7.0-2021-07-21
drwxr-xr-x 4 root root 4096 Jan 19 05:12 zindi

Content of /content/data/zindi:
total 556
drwxr-xr-x 3 root root   4096 Jan 19 05:11 __MACOSX
drwxr-xr-x 2 root root 266240 Oct 22 06:14 test_audio
-rw------- 1 root root 292195 Jan 19 05:12 Test.csv


## Model Training

We will fine-tune the Facebook Wav2Vec2 model with the Luganda Common Voice dataset.

In [10]:
%%time

%cd /content/luganda-asr

!mkdir -p output
!python run_finetuning.py finetuning_common_voice_1epoch.json

/content/luganda-asr
01/19/2022 05:19:05 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
01/19/2022 05:19:05 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=8,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=2000,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=2,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=True,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
labe

## Evaluation

### Evaluation on Common Voice version 6.1

We evaluate the ASR model "indonesian-nlp/wav2vec2-luganda" on the test split of Common Voice Dataset for Luganda (version 6.1). The evaluation also uses the Ken Language Model (KenLM) 5gram.bin. It takes around 9 minutes. We get here the Word Error Rate (WER) of **7.37%**.

In [32]:
%%time

!python run_evaluation.py -m indonesian-nlp/wav2vec2-luganda -n common_voice -c lg -k 5gram.bin --test_pct 100

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Downloading: 100% 1.18G/1.18G [00:32<00:00, 38.5MB/s]
Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
No known unigrams provided, decoding results might be a lot worse.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/lg/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)
100% 584/584 [00:08<00:00, 70.98ex/s]
100% 584/584 [01:31<00:00,  6.37ex/s]
Downloading: 4.49kB [00:00, 773kB/s]        
tcmalloc: large alloc 1253105664 bytes == 0x55aea7cd0000 @  0x7f944d519615 0x55ad576814cc 0x55ad5776147a 0x55ad576842ed 0x55ad57684240 0x55ad576f7973 0x55ad57685afa 0x55ad576f7d00 0x55ad57685afa 0x55ad576f3915 0x55ad576f29ee 0x55ad57685bda 0x55ad576f3915 0x55ad57685afa 0x55ad576f3915 0x55ad576f29ee 0x55ad57685bda 0x55ad576f3915 0x55ad576f29ee 0x55ad5768648

### Evaluation on Common Voice version 7.0

We evaluate the ASR model "indonesian-nlp/wav2vec2-luganda" on the test split of Common Voice Dataset for Luganda (version 7.0). The evaluation also uses the Ken Language Model (KenLM) 5gram.bin. It takes around 25 minutes. We get here the Word Error Rate (WER) of **7.53%**.

In [54]:
%%time

!python run_evaluation.py -m indonesian-nlp/wav2vec2-luganda -n ./dataset/common_voice -c lg --data_dir /content/data/cv-corpus-7.0-2021-07-21 -k 5gram.bin --test_pct 100

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
No known unigrams provided, decoding results might be a lot worse.
Using custom data configuration lg-b5dd5bec651497f8
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/lg-b5dd5bec651497f8/7.0.0/d2815414a38db497e900b5788f75958a15449c23acb8371ded02d3ed746d6206)
100% 4276/4276 [00:48<00:00, 88.69ex/s] 
100% 4276/4276 [02:15<00:00, 31.65ex/s]
tcmalloc: large alloc 1253105664 bytes == 0x5590e86d8000 @  0x7f08bad88615 0x558fbd8524cc 0x558fbd93247a 0x558fbd8552ed 0x558fbd855240 0x558fbd8c8973 0x558fbd856afa 0x558fbd8c8d00 0x558fbd856afa 0x558fbd8c4915 0x558fbd8c39ee 0x558fbd856bda 0x558fbd8c4915 0x558fbd856afa 0x558fbd8c4915 0x558fbd8c39ee 0x558fbd856bda 0x558fbd8c4915 0x558fbd8c39ee 0x558fbd85748c 0x558fbd857698 0x558fbd

## Submission

Creating the submission file submissions/luganda-asr.csv
using the ASR model "indonesian-nlp/wav2vec2-luganda" on the test set
provided by Zindi.
It takes around 40 minutes

In [51]:
%%time

!mkdir -p submissions
# For testing purpose, create submission file for 1% of test dataset
#!python run_submission.py -o submissions/luganda-asr.csv -m indonesian-nlp/wav2vec2-luganda --data_dir /content/data/zindi -k 5gram.bin --test_pct 1

# create submission file for 100% of test dataset
!python run_submission.py -o submissions/luganda-asr.csv -m indonesian-nlp/wav2vec2-luganda --data_dir /content/data/zindi -k 5gram.bin



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
No known unigrams provided, decoding results might be a lot worse.
Using custom data configuration lg-e6ac6d8c877e86bb
Reusing dataset zindi (/root/.cache/huggingface/datasets/zindi/lg-e6ac6d8c877e86bb/1.0.0/48535cc6e254ea4fe8bd529ebd18452c88df0776030e5c69a33ea03b6bcf7436)
100% 71/71 [00:00<00:00, 104.72ex/s]
  0% 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1093: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor